# Import Lib

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import time
from sklearn.cluster import KMeans
import pickle

from scipy.spatial.distance import cdist

In [2]:
list_image_path =[]
list_image = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path =os.path.join(dirname, filename)
        list_image_path.append(path)
        image = cv2.imread(path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
        list_image.append(image)

In [3]:
# Trích suất đặc trưng bằng SIFT
def extract_sift_feature(all_image):
    image_descriptors = []
    for i in range(len(all_image)):
        sift = cv2.SIFT_create()
        _,des = sift.detectAndCompute(all_image[i],None)
        for j in des:
            image_descriptors.append(j) 
    return image_descriptors

In [4]:
def extract_sift_feature_single(image):
    sift =cv2.SIFT_create()
    _,des = sift.detectAndCompute(image,None)
    return des

In [5]:
def kmeans_bow(image_descriptors, num_clusters):
    strar = time.time()
    bow_dict = []
    kmeans = KMeans(n_clusters=num_clusters, verbose = 1).fit(all_descriptors)
    bow_dict = kmeans.cluster_centers_
    
    return bow_dict

In [6]:
all_descriptors = extract_sift_feature(list_image)

In [7]:
num_clusters = 50
if not os.path.isfile('bow_dictionary.pkl'):
    BoW = kmeans_bow(all_descriptors, num_clusters)
    pickle.dump(BoW, open('bow_dictionary.pkl', 'wb'))
else:
    BoW = pickle.load(open('bow_dictionary.pkl', 'rb'))

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Initialization complete
Iteration 0, inertia 1259995113.0.
Iteration 1, inertia 855415295.5745232.
Iteration 2, inertia 835711724.1473672.
Iteration 3, inertia 826394381.6177311.
Iteration 4, inertia 819596416.7955182.
Iteration 5, inertia 814578808.2353237.
Iteration 6, inertia 811461579.6348026.
Iteration 7, inertia 809707120.5958617.
Iteration 8, inertia 808586904.4690537.
Iteration 9, inertia 807658207.274205.
Iteration 10, inertia 806790799.9796106.
Iteration 11, inertia 806033012.5211022.
Iteration 12, inertia 805258267.7173381.
Iteration 13, inertia 804580849.895048.
Iteration 14, inertia 803835457.6972661.
Iteration 15, inertia 803108679.4162828.
Iteration 16, inertia 802512853.3002597.
Iteration 17, inertia 801964539.1722858.
Iteration 18, inertia 801450741.5200478.
Iteration 19, inertia 801050300.5815011.
Iteration 20, inertia 800757341.7455329.
Iteration 21, inertia 800538889.4270375.
Iteration 22, inertia 800437151.5236819.
Iteration 23, inertia 800355483.0490658.
Iteration

In [8]:
print(len(BoW))

50


In [9]:
def create_features_bow(single_image_descriptors, BoW, num_clusters):
    feature = np.array([0] * num_clusters)
    if single_image_descriptors is not None:
        distance = cdist(single_image_descriptors, BoW, metric='euclidean')
        argmin = np.argmin(distance, axis=1)
        for j in argmin:
            feature[j] += 1
    return feature


In [10]:
tmp = extract_sift_feature_single(list_image[0])
print(tmp.shape)
feature_bow = create_features_bow(tmp,BoW,num_clusters)
print(feature_bow)

(31, 128)
[4 0 1 0 0 0 0 0 0 0 4 0 0 0 1 0 0 0 0 0 0 0 0 0 0 7 0 0 1 0 1 1 1 0 1 2 0
 0 0 0 0 0 1 0 0 0 2 0 4 0]
